In [3]:
! pip install  mlxtend pandas numpy

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Imports

In [9]:
from mlxtend.frequent_patterns import fpgrowth, association_rules, apriori
import pandas as pd
import numpy as np
from itertools import combinations
import time

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Code

In [10]:
class AssociationRulesGenerator:
    def __init__(self, min_confidence, min_support):
        self.conf = min_confidence
        self.min_supp = min_support

    def read_data_set(self,transactions_file_path):
        """
        Read CSV file
        Args:
            transactions_file_path (str): Path to the CSV file

        Returns:
            pd.DataFrame: One-hot-encoded DataFrame
        """
        items_transactions_df = pd.read_csv(transactions_file_path)
        items_transactions_df = items_transactions_df.fillna(0)
        return items_transactions_df

    def brute_force(self, items_df):
        """This function returns frequent items using brute force technique

        Args:
            items_df (DataFframe): Items Transactiion dataframe

        Returns:
            Dataframe: Frequent items
        """


        # Generate itemsets of size 1 with their support
        fq_item_set1 = {k: round(items_df[k].sum() / len(items_df), 3) for k in items_df.columns}
        freq_item_set_df = pd.DataFrame(list(fq_item_set1.items()), columns=["itemsets", "support"])
        selected_keys = list(fq_item_set1.keys())

        print(f"Item Set 1 \n{freq_item_set_df}")
        item_set_count = 2

        while len(selected_keys) != 0:
            items_combinations = combinations(selected_keys, item_set_count)
            support_val_dict = {}
            for item_key in items_combinations:
                t = np.prod(items_df.loc[:, item_key].values, axis=1)
                support_val_dict[item_key] = round(sum(t == 1) / len(items_df), 3)

            #break if support values dictionary is empty
            if len(support_val_dict) == 0:
                selected_keys = []
                break

            item_set_df = pd.DataFrame(list(support_val_dict.items()), columns=["itemsets", "support"])

            if len(item_set_df[item_set_df['support'] >= self.min_supp]) == 0:
                selected_keys = []
                break

            #Printtingeach item set
            print(f"Items Set {item_set_count}\n{item_set_df}")
            freq_item_set_df = pd.concat([freq_item_set_df, item_set_df])
            item_set_count += 1

        #Checking Support
        freq_item_set_df = freq_item_set_df[freq_item_set_df['support'] >= self.min_supp].reset_index(drop=True)
        print(f"Final Item Sets: \n{freq_item_set_df}")
        freq_item_set_df['itemsets'] = [k if isinstance(k, tuple) else (k,) for k in freq_item_set_df.itemsets]
        return freq_item_set_df

    def apriori(self, items_df):
        """
        Apriori algorithm.
        """
        print("Frequency Item Set Apriori Algorithm")
        freq_item_set_df = apriori(items_df, min_support=self.min_supp, use_colnames=True)
        print(f"Final Items Set : \n{freq_item_set_df}")
        return freq_item_set_df

    def fp_growth(self, items_df):
        """
         FP-growth algorithm.
        """
        print("Frequency Item Set using FP-Growth Algorithm")
        freq_item_set_df = fpgrowth(items_df, min_support=self.min_supp, use_colnames=True)
        print(f"Final Items Set: \n{freq_item_set_df}")
        return freq_item_set_df

    def comparions(self,bf_t,ap_t,fp_t):
        print(f"Apriori is {bf_t//ap_t} times faster than brute force ")
        print(f"FP-Growth is {bf_t//fp_t} times faster than brute force ")
        print(f"FP-Growth is {ap_t//fp_t} time faster than Apriori")



    def rule_miner(self, item_file_path):

        data = self.read_data_set(item_file_path)

        print(f"Executing using Brute Force Algorithm")
        bf_st = time.time()
        bruteforce_items_support_df = self.brute_force(data)
        brute_force_rules = association_rules(bruteforce_items_support_df, metric="confidence", min_threshold=self.conf)
        print("Brute Algorithm_rules")
        for i, rule in brute_force_rules.iterrows():
            print(f"Rule {i+1}:  {', '.join(rule['antecedents'])} => {', '.join(rule['consequents'])}; confidence: {round(rule['confidence'], 3)}")
        bf_et = time.time()
        bf_t = bf_et-bf_st
        print(f"Brute_force execution time : {round(bf_t,3)}")


        print("Executing Apriori Algorithm")
        ap_st = time.time()
        apriori_items_support_df = self.apriori(data)
        apriori_rules = association_rules(apriori_items_support_df, metric="confidence", min_threshold=self.conf)
        apriori_rules = apriori_rules[['antecedents', 'consequents', 'confidence']]
        print("Apriori Rules")
        for i, rule in apriori_rules.iterrows():
            print(f"Rule {i+1}:  {', '.join(rule['antecedents'])} => {', '.join(rule['consequents'])}; confidence: {round(rule['confidence'], 3)}")
        ap_et = time.time()
        ap_t = ap_et - ap_st
        print(f"Apriori execution time : {round(ap_t,3)}")

        print("Executiong FP-growth Algorithm")
        fp_st = time.time()
        fp_growth_items_support_df = self.fp_growth(data)
        fp_growth_rules = association_rules(fp_growth_items_support_df, metric="confidence", min_threshold=self.conf)
        fp_growth_rules = fp_growth_rules[['antecedents', 'consequents', 'confidence']]
        print("FP-Growth Rules:")
        for i, rule in apriori_rules.iterrows():
            print(f"Rule {i+1}:  {', '.join(rule['antecedents'])} => {', '.join(rule['consequents'])}; confidence: {round(rule['confidence'], 3)}")
        fp_et = time.time()
        fp_t = fp_et -fp_st
        print(f"FP-Growth Execution time : {round(fp_t,3)}")

        self.comparions(bf_t=bf_t, ap_t=ap_t, fp_t=fp_t)

def main():
    datasets = {
        1: "Data/Items1.csv",
        2: "Data/Items2.csv",
        3: "Data/Items3.csv",
        4: "Data/Items4.csv",
        5: "Data/Items5.csv"
    }

    for dataset_selection, item_file_path in datasets.items():
        print(f"\nProcessing Dataset {dataset_selection}: {item_file_path}")


        min_support = float(input(f"Enter min_threshold_support (in %): ")) / 100
        min_confidence = float(input(f"Enter min_threshold_confidence  (in %): ")) / 100

        association_rules_generator = AssociationRulesGenerator(min_confidence, min_support)

        association_rules_generator.rule_miner(item_file_path)

print("Mining Completed")


if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Mining Completed

Processing Dataset 1: Data/Items1.csv
Enter min_threshold_support (in %): 30
Enter min_threshold_confidence  (in %): 30


/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Executing using Brute Force Algorithm
Item Set 1 
     itemsets  support
0        Eggs     0.65
1        Milk     0.45
2       Bread     0.50
3      Butter     0.50
4       Apple     0.40
5      Banana     0.50
6     Chicken     0.35
7   Lamb Meat     0.35
8        Soda     0.40
9        Wine     0.25
10      Flour     0.40
Items Set 2
                itemsets  support
0           (Eggs, Milk)     0.30
1          (Eggs, Bread)     0.30
2         (Eggs, Butter)     0.35
3          (Eggs, Apple)     0.15
4         (Eggs, Banana)     0.25
5        (Eggs, Chicken)     0.25
6      (Eggs, Lamb Meat)     0.25
7           (Eggs, Soda)     0.30
8           (Eggs, Wine)     0.20
9          (Eggs, Flour)     0.25
10         (Milk, Bread)     0.25
11        (Milk, Butter)     0.25
12         (Milk, Apple)     0.10
13        (Milk, Banana)     0.15
14       (Milk, Chicken)     0.10
15     (Milk, Lamb Meat)     0.15
16          (Milk, Soda)     0.15
17          (Milk, Wine)     0.05
18         (Milk

/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(



Rule 2:  Butter => Eggs; confidence: 0.667
Rule 3:  Eggs => Apple; confidence: 0.571
Rule 4:  Apple => Eggs; confidence: 0.727
Rule 5:  Banana => Eggs; confidence: 0.625
Rule 6:  Eggs => Chicken; confidence: 0.5
Rule 7:  Chicken => Eggs; confidence: 0.778
Rule 8:  Lamb Meat => Eggs; confidence: 0.636
Rule 9:  Eggs => Lamb Meat; confidence: 0.5
Rule 10:  Soda => Eggs; confidence: 0.75
Rule 11:  Flour => Eggs; confidence: 0.75
Rule 12:  Milk => Butter; confidence: 0.625
Rule 13:  Butter => Milk; confidence: 0.556
Rule 14:  Milk => Banana; confidence: 0.625
Rule 15:  Banana => Milk; confidence: 0.625
Rule 16:  Milk => Lamb Meat; confidence: 0.875
Rule 17:  Lamb Meat => Milk; confidence: 0.636
Rule 18:  Bread => Apple; confidence: 0.875
Rule 19:  Apple => Bread; confidence: 0.636
Rule 20:  Bread => Chicken; confidence: 0.875
Rule 21:  Chicken => Bread; confidence: 0.778
Rule 22:  Soda => Bread; confidence: 0.625
Rule 23:  Bread => Soda; confidence: 0.625
Rule 24:  Butter => Banana; confid

/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Final Items Set: 
     support                      itemsets
0       0.60                   (Lamb Meat)
1       0.55                       (Flour)
2       0.55                       (Bread)
3       0.50                        (Eggs)
4       0.45                      (Banana)
..       ...                           ...
118     0.30  (Butter, Lamb Meat, Chicken)
119     0.20   (Chicken, Lamb Meat, Flour)
120     0.25                (Milk, Butter)
121     0.20                 (Milk, Flour)
122     0.20                 (Milk, Bread)

[123 rows x 2 columns]
FP-Growth Rules:
Rule 1:  Apple => Eggs; confidence: 0.857
Rule 2:  Eggs => Lamb Meat; confidence: 0.8
Rule 3:  Milk => Butter; confidence: 0.714
Rule 4:  Banana => Bread; confidence: 0.889
Rule 5:  Bread => Banana; confidence: 0.727
Rule 6:  Wine => Bread; confidence: 0.833
Rule 7:  Butter => Lamb Meat; confidence: 0.818
Rule 8:  Lamb Meat => Butter; confidence: 0.75
Rule 9:  Wine => Butter; confidence: 0.833
Rule 10:  Apple => Lamb Meat

/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [ ]:
3